# Ollaman käyttö

-Valtteri Airaksinen

---

### Virtuaaliympäristön luonti ja käyttö

Virtuaaliympäristön luonti:

`python -m venv llama`

Ympäristön käyttöönotto:

`source llama/Scripts/activate`

Pakettien lataaminen:

`pip install -r requirements.txt`

Ympäristön käytöstä poisto:

`deactivate`

---

### Ollaman käyttö

Käynnistys:

`ollama serve`

Haetaan mallit:

Avaa uusi terminaali -> Ota ympäristö käyttöön

Nyt voit valita minkä mallin haluat ja noutaa sen esimerkiksi komennolla `ollama pull llama3.2:3b`

Malleja:

`llama3.2:1b` - 1.3GB

`llama3.2:3b` - 2GB

`llama3.1:8b` - 4.7GB

`llama3.1:70b` - 40GB

`llama3.1:405b` - 229GB

`llama3:8b` - 4.7GB

`llama3:70b` - 40GB

Jos mallia ei spesifioi, noudetaan tuorein malli esim `llama3.2` -> `llama3.2:3b`

Näytä ladatut mallit:

`ollama list`

---

In [ ]:
from langchain_community.llms import Ollama

# Specify which model you want to use
llm = Ollama(model="llama3:latest")

# Chatbot
print(llm.invoke("Ymmärrätkö jos annan sinulle käsiteltäväksi suomenkielistä dataa jonka haluan sinun jaoittelevan?", stop=['<|eot_id|>']))


### Suomenkielen ymmärrys

Kysymys: Ymmärrätkö jos annan sinulle käsiteltäväksi suomenkielistä dataa jonka haluan sinun jaoittelevan?

---

`llama3.2:1b` Ehko, en ole yhtynyt kyseessä olevaan suomen kielen ja taidetta koskevaan tuotannoon. Tietoa on ainoastaan sijoiteltu lähemmäs sinua kuin muille joutuneille.

Tietoja ei ole vielä saatavilla, mutta oletettavasti voisin tarkistaa tietoa sitten kunhan käynnistyisi tehtäviä tai pyrkimyksiä. (14.5s)

---

`llama3.2:3b` Olen valmis käsittelemään suomenkielistä dataa! Olen koulutettu käsitelleämään erilaisia kieli- ja tekniikkatekijoita, mutta varsinkin suomalaisen kielen ja sen käyttöjärjestelmien ymmärrystä on olemassa vaikeuksia.

Käytän aina kielitietokantoja tai -tietoa esimerkiksi Wikipedia-sta, Valtion tutustumislainausverkkon tiedostoista, tai muuta luottavaa lähteerää. Kun sinulla on suomenkielinen data, olen valmis käsittelemään sitä.

Onko dataa kosketus- tai teksti-tyyppinen? Mikä on tarkoitusta? (28.6s)

---

`llama3:8b` Hei!

Kyseessä on suomalainen chatbot, joka on valmiina vastaanottamaan ja käsittämään suomenkielisiä tilastotietoja. Olen rakennettu oppimaan jaymmärtämään erilaisia kieliä, myös suomea.

Kuten haluat, anna minulle käsiteltäväksi suomenkielistä dataa, ja minä yritän tehdä siitä järkevää. Olkoon tietoa miten hyvin minä kykenen käsittämään sen! (46.2s)

### Paranneltuja versioita funktiosta

In [ ]:
from langchain_community.llms import Ollama

# Define the reusable portion of the prompt
prompt_template = """
Extract the following information from the text below, the text is in finnish:
- Maalaji
- Multavuus
- pH
- Nurmen ikä
- Mitä viljellään
- Lannoite
- Lannoitemäärä

Some helpful information:

Here are all maalaji's, maalaji will always be one or multiple of these, it can also be None if not mentioned:
    "AS   aitosavi",
    "BCt  ruskosammalsaraturve",
    "CSt  sararahkaturve",
    "Ct   saraturve",
    "He   hiue",
    "HeS  hiuesavi",
    "HHk  hieno hiekka",
    "HHt  hieno hieta",
    "HkMr hiekka moreeni",
    "Hs   hiesu",
    "HsMr hiesumoreeni",
    "HsS  hiesusavi",
    "Ht   hieta",
    "HtMr hietamoreeni",
    "HtS  hietasavi",
    "Jm   järvimuta",
    "KHk  karkea hiekka",
    "KHt  karkea hieta",
    "LCt  metsäsaraturve",
    "Lj   lieju",
    "LjS  liejusavi, urpasavi",
    "LSt  metsärahkaturve",
    "Mm   multamaa",
    "Mt   muta",
    "SCt  rahkasaraturve",
    "SMr  savimoreeni",
    "sHHt savinen hienohieta,
    "Sr   sora",
    "SrMr soramoreeni",
    "St   rahkaturve"

These are the different multavuus values, multavuus will always be one or multiple of these, it can also be None if not mentioned:
    vm    vähämultainen  
    m     multava  
    rm    runsasmultainen  
    erm   erittäin runsasmultainen  
    Mm    multamaa  

Multavuus can be close to maalaji written together with it, for example rmHtMr. In this case:
    Maalaji: HtMr Hietamoreeni
    Multavuus: rm Runsasmultainen
    
Answer to "Mitä viljellään?" is often a list of items with percentages or kg amounts on how much of them is in the field, for example:
    TT Tuure 35,5%, TT Rhonia 24,5%, NN Klaara 15%, RN Karolina 5%, Engl.rh Riikka10%, PA Saija 7,5%, VA Silvester 2,5%
    Here TT is Timotei, NN is Nurminata, RN is Ruokonata, Engl.rh is Englannin raiheinä, PA is Puna-apila, VA is Valko-apila and AA is Alsike-apila.
    Tuure, Rhonia, Klaara, Karolina, Riikka, Saija & Silvester are the brand names. Brand names might also not be specified for example:
    TT 20%, RN 80%
There can also be just a single value like "Valio Carbo nurmiseos" or "Etelän paalinurmiseos"
Always specify the kind of item is (TT, RN, VA, etc.) and the amount if those are mentioned

Nurmen ikä is often a number and v or vuosi close to word nurmi, for example 2 v. nurmi. In this case:
    Nurmen ikä: 2v

Lannoitus and lannoitusmäärä:
    for example from input: Lannoitus 14.5. YaraMila Y3 400 kg/ha.
    14.5 is the date (not to be extracted)
    Lannoitus: YaraMila Y3
    Lannoitusmäärä: 400 kg/ha

    Another example:  20.4. Se-salpietari 350 kg/ha.
    20.4. is the date (not to be extracted)
    Lannoitus: Se-salpietari
    Lannoitusmäärä: 350 kg/ha

Some of the values we are looking for can be missing or have typos in them.
If values are not found do not fill them in, just put None.
Dont make any assumptions, if no value for a field can be found just put None

Provide the output in JSON format as shown below:

{
    "Maalaji": "[extracted maalaji]",
    "Multavuus": "[extracted multavuus]",
    "pH": "[extracted pH]",
    "Nurmen ikä": "[extracted nurmen ikä]",
    "Mitä viljellään": "[extracted mitä viljellään]",
    "Lannoite": "[extracted lannoite]",
    "Lannoitemäärä": "[extracted lannoitemäärä]"
}

Here is a few examples:
Input: TS Laatunurmi: TT Rubinia 40% + Tenho 25%, NN Valtteri 20%, engl.rh Riikka 10% + Mathilde 5%, lisätty RN 5%.  Lannoitus 13.5. Belor Premium (22-2-7-S3). Maalaji rm HtMr. 2v. nurmi

Output:
{
    "Maalaji": "Hietamoreeni",
    "Multavuus": "Runsasmultainen",
    "pH": null,
    "Nurmen ikä": "2v",
    "Mitä viljellään": "TS Laatunurmi: TT Rubinia 40% + Tenho 25%, NN Valtteri 20%, engl.rh Riikka 10% + Mathilde 5%, lisätty RN 5%",
    "Lannoite": "Belor Premium (22-2-7-S3)",
    "Lannoitemäärä": null
}

Input: Hankkija Laatu 50% ja Tuure timotei 50%. 370kg/ha Y25 ja liete 25tn/ha. Htmr. Antinhaikara

Output:
{
    "Maalaji": "Hietamoreeni",
    "Multavuus": null,
    "pH": null,
    "Nurmen ikä": null,
    "Mitä viljellään": "Hankkija Laatu 50% ja Tuure timotei 50%",
    "Lannoite": "Y25, liete",
    "Lannoitemäärä": "370 kg/ha, 25 tn/ha"
}

Input: 3v nurmi rm HiueSavella. pH 6,1.  Perust Tilasiemenen apilaseos. Lann. Belor Premium 27 Se 360 kg, eli 97 kg N/ha. 

Output:
{
    "Maalaji": "Hiuesavi",
    "Multavuus": "Runsasmultainen",
    "pH": 6.1,
    "Nurmen ikä": "3v",
    "Mitä viljellään": "Perust Tilasiemenen apilaseos",
    "Lannoite": "Belor Premium 27 Se",
    "Lannoitemäärä": "360 kg, 97 kg N/ha"
}

Input: Korjuu 16.6. Tuure 85%, Nurminata 5%, PA 10%. Lannoitus YaraMila NK1 440 kg/ha.

Output:
{
    "Maalaji": null,
    "Multavuus": null,
    "pH": null,
    "Nurmen ikä": null,
    "Mitä viljellään": "Tuure 85%, Nurminata 5%, PA 10%",
    "Lannoite": "YaraMila NK1",
    "Lannoitemäärä": "440 kg/ha"
}

Input: Näytepaikka: Ylistaro. TT Tenho 25% ja Nuutti 25%, NN Fure 20%, Engl.RH Mathilde 15%, RN Kora 10 %, Rainata Hykor 5%. Lannoitus:Yara Mila Y25 440 kg/ha. Maalaji m sHHt . Perustettu 2018

Output:
{
    "Maalaji": "Savinen Hieno hieta",
    "Multavuus": "Multava",
    "pH": null,
    "Nurmen ikä": null,
    "Mitä viljellään": "TT Tenho 25% ja Nuutti 25%, NN Fure 20%, Engl.RH Mathilde 15%, RN Kora 10 %, Rainata Hykor 5%",
    "Lannoite": "YaraMila Y25",
    "Lannoitemäärä": "440 kg/ha"
}

Input: "nan"

Output:
{
"Maalaji": null,
"Multavuus": null,
"pH": null,
"Nurmen ikä": null,
"Mitä viljellään": null,
"Lannoite": null,
"Lannoitemäärä": null
}

Input: Näytepaikka Luopajärvi. 2v. nur TT Tuure 35% + Rhonia 25%, NN Klaara 20%, RN Karolina 10%, engl.rh Riikka 10%. lannoitus  6.5. YaraMila Y10 380 kg/ha + YaraLiva Nitrabor N15 130 kg/ha. Maalaji rm HHt

Output:
{
"Maalaji": "Hieno hieta",
"Multavuus": Runsasmultainen,
"pH": null,
"Nurmen ikä": "2v",
"Mitä viljellään": "TT Tuure 35% + Rhonia 25%, NN Klaara 20%, RN Karolina 10%, engl.rh Riikka 10%",
"Lannoite": "YaraMila Y10, YaraLiva Nitrabor N15",
"Lannoitemäärä": "380 kg/ha, 130 kg/ha"
}

Ensure that each field is included, even if the value is null

No extra information is required from you, just the output in the requested format.
"""

In [ ]:
# Function for extracting the values
# ~50% faster than the original
def extractValuesFromText(llm, df):
    for index, row in df.iterrows():
        text = row['INFOTEKSTI']
        
        # Combine the reusable template with the input text
        full_prompt = f"{prompt_template}\n\nText: \"{text}\""

        # Invoke the model with the combined prompt
        response = llm.invoke(full_prompt)

        # Print the result
        print(f"Row {int(index) + 1}")
        print("Input text: ", text)
        print("")
        print("Extracted Data: ", response)
        print("")


In [11]:
import re
import pandas as pd
import json

# Function for batch processing
def extractValuesFromTextBatch(llm, df, batch_size=5):
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]
        texts = batch['INFOTEKSTI'].tolist()

        # Create a single prompt for all rows in the batch
        batch_prompt = prompt_template + "\n\n"
        for idx, text in enumerate(texts):
            batch_prompt += f"Text {idx + 1}: \"{text}\"\n"

        # Add a clear instruction for batch output formatting
        batch_prompt += "\nProvide output in the format:\n"
        batch_prompt += "\n".join(
            [f"Text {j + 1}: Input: \"{{Original text}}\" \nExtracted Data: {{Extracted data}}" for j in range(len(texts))]
        )

        # Invoke the model with the batched prompt
        response = llm.invoke(batch_prompt)

        # Print the result
        print(f"Batch {i // batch_size + 1} Results:")
        print(response)
        print("")

        # Step 1: Extract the JSON-like blocks
        json_blocks = re.findall(r"Extracted Data:\s*({.*?})", response, re.DOTALL)

        # Step 2: Parse JSON and load into a list of dictionaries
        parsed_data = [json.loads(block) for block in json_blocks]

        # Step 3: Create a DataFrame
        df = pd.DataFrame(parsed_data)
        return df


In [40]:
# Function for batch processing and adding extracted values to the DataFrame
def extractValuesFromTextBatch(llm, df, batch_size=5):
    # Initialize columns for extracted values
    columns = ["Maalaji", "Multavuus", "pH", "Nurmen ikä", "Mitä viljellään", "Lannoite", "Lannoitemäärä"]
    for col in columns:
        if col not in df.columns:
            df[col] = None  # Add empty columns to the DataFrame if not present

    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]
        texts = batch['INFOTEKSTI'].tolist()

        # Create a single prompt for all rows in the batch
        batch_prompt = prompt_template + "\n\n"
        for idx, text in enumerate(texts):
            batch_prompt += f"Text {idx + 1}: \"{text}\"\n"

        # Add a clear instruction for batch output formatting
        batch_prompt += "\nProvide output in the format:\n"
        batch_prompt += "\n".join(
            [f"Text {j + 1}: Input: \"{{Original text}}\" Extracted Data: {{Extracted data}}" for j in range(len(texts))]
        )

        # Invoke the model with the batched prompt
        response = llm.invoke(batch_prompt)

        # Debug: Print raw response for inspection
        print(f"Batch {i // batch_size + 1} Results:")
        print(response)
        print("")

        # Parse the model response
        responses = response.split("\nText ")  # Split based on 'Text X:' markers
        for line in responses:
            if not line.strip() or "Input:" not in line:  # Skip empty or malformed responses
                continue

            # Extract text number and split into input and extracted data
            try:
                text_id = int(line.split(":")[0])  # Extract text number (e.g., '1' from 'Text 1')
                extracted_data_block = line.split("Extracted Data:")[-1].strip()
            except (ValueError, IndexError):
                continue  # Skip malformed lines

            # Extract individual fields
            extracted_values = {}
            for col in columns:
                start = f"{col}: "
                if start in extracted_data_block:
                    value = extracted_data_block.split(start, 1)[-1].split("\n", 1)[0].strip()
                    extracted_values[col] = value if value.lower() != 'none' else None
                else:
                    extracted_values[col] = None

            # Update DataFrame for the corresponding row
            row_idx = batch.index[text_id - 1]  # Match batch index to DataFrame row
            print(f"Updating row {row_idx} with: {extracted_values}")  # Debug print

            # Only update if the new value is not None
            for col in columns:
                if extracted_values[col] is not None:  # Check if the value is not None
                    df.at[row_idx, col] = extracted_values[col]

    return df


In [2]:
import json

# Function for batch processing and adding extracted values to the DataFrame
def extractValuesFromTextBatch(llm, df, batch_size=5):
    # Initialize columns for extracted values
    columns = ["Maalaji", "Multavuus", "pH", "Nurmen ikä", "Mitä viljellään", "Lannoite", "Lannoitemäärä"]
    for col in columns:
        if col not in df.columns:
            df[col] = None  # Add empty columns to the DataFrame if not present

    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]
        texts = batch['INFOTEKSTI'].tolist()

        # Create a single prompt for all rows in the batch
        batch_prompt = prompt_template + "\n\n"
        for idx, text in enumerate(texts):
            batch_prompt += f"Text {idx + 1}: \"{text}\"\n"

        # Add a clear instruction for batch output formatting in JSON
        batch_prompt += "\nProvide output in JSON format for each text:\n"
        batch_prompt += "\n".join(
            [f"Text {j + 1}: {{JSON object}}" for j in range(len(texts))]
        )

        # Invoke the model with the batched prompt
        response = llm.invoke(batch_prompt)

        # Debug: Print raw response for inspection
        print(f"Batch {i // batch_size + 1} Results:")
        print(response)
        print("")

        # Parse the model response as JSON
        try:
            parsed_responses = response.split("\nText ")
            for line in parsed_responses:
                if not line.strip() or "{" not in line:  # Skip empty or malformed lines
                    continue

                try:
                    text_id = int(line.split(":")[0])  # Extract text number (e.g., '1' from 'Text 1')
                    json_data = line.split(":", 1)[-1].strip()
                    extracted_data = json.loads(json_data)  # Parse JSON data
                except (ValueError, json.JSONDecodeError):
                    print(f"Failed to parse line: {line}")  # Debug message for faulty lines
                    continue  # Skip this line if parsing fails

                # Update DataFrame for the corresponding row
                row_idx = batch.index[text_id - 1]  # Match batch index to DataFrame row
                print(f"Updating row {row_idx} with: {extracted_data}")  # Debug print

                # Only update if the new value is not None
                for col in columns:
                    if col in extracted_data and extracted_data[col] is not None:  # Check if the key exists and is not None
                        df.at[row_idx, col] = extracted_data[col]
        except Exception as e:
            print(f"Error processing batch {i // batch_size + 1}: {e}")

    return df


In [12]:
import pandas as pd

# Creating very tiny test sample
df = pd.read_excel('data/Korjuuaikanäytteiden_lisätiedot_16082024.xlsx')
#test_sample = df.head(5)
#test_sample = df.tail(5)
test_sample = df.sample(n=5)
test_sample.reset_index(inplace=True)

In [13]:
from pathlib import Path
from string import Template

question_file = Path("data/prompt.md")
prompt_template = Template(question_file.read_text())
prompt_template = prompt_template.safe_substitute()
prompt_template

'You are a highly intelligent system capable of converting human-written descriptions of travel experiences into structured JSON format. Write out the JSON and nothign but the JSON. Each description includes details about the field - but any of these can also be missing. The text is in finnish. Your task is to extract the key pieces of information and represent them in JSON format with the following fields:\n\n    Maalaji: One of the values in the "Maalajit" -list if known, else null.\n    Multavuus: One of the values in the "Multavuudet" -list if known, else null.\n    pH: Float value representing pH -value.\n    Nurmen ikÃ¤: Age of the grass, is often a number and v or vuosi close to word nurmi.\n    MitÃ¤ viljellÃ¤Ã¤n: Often a list of items with percentages or kg amounts on how much of them is in the field, can also be a single value like "Valio Carbo nurmiseos" or "EtelÃ¤n paalinurmiseos"\n    Lannoite: Fertilizer.\n    LannoitemÃ¤Ã¤rÃ¤: Amount of fertilizer used.\n\nMaalajit:\n   

In [14]:
from langchain_community.llms import Ollama

# Define the model you are using
llm = Ollama(model="llama3:latest")  # You can choose the model you want

# Run the function
#extractValuesFromText(llm, test_sample)
df = extractValuesFromTextBatch(llm, test_sample, 5)
df

Batch 1 Results:
Here are the outputs:

Text 1:
Input: "TT Rhonia 30%, Tuure 30%, NN Ilmari 20 %"
Extracted Data: {
    "Maalaji": null,
    "Multavuus": null,
    "pH": null,
    "Nurmen ikÃ¤": null,
    "MitÃ¤ viljellÃ¤Ã¤n": "TT Rhonia 30%, Tuure 30%, NN Ilmari 20 %",
    "Lannoite": null,
    "LannoitemÃ¤Ã¤rÃ¤": null
}

Text 2:
Input: "multamaan säilörehunurmi (K-maatalouden myymä seos) + n 20% puna-apilaa,  19.5.2017 Yaramila Y20  420kg/ha"
Extracted Data: {
    "Maalaji": null,
    "Multavuus": null,
    "pH": null,
    "Nurmen ikÃ¤": null,
    "MitÃ¤ viljellÃ¤Ã¤n": "multamaan säilörehunurmi (K-maatalouden myymä seos) + n 20% puna-apilaa",
    "Lannoite": "YaraMila Y20",
    "LannoitemÃ¤Ã¤rÃ¤": "420 kg/ha"
}

Text 3:
Input: "TT Grinstad 50%, Uula 20%, NN Valtteri 10%, RN Karolina 10%, engl.rh. Riikka 10%. Lannoitus 29.4. YaraMila Y3 450 kg/ha. Maalaji m HtMr"
Extracted Data: {
    "Maalaji": "mHtMr",
    "Multavuus": null,
    "pH": null,
    "Nurmen ikÃ¤": null,
    "MitÃ¤ viljel

,Maalaji,Multavuus,pH,Nurmen ikÃ¤,MitÃ¤ viljellÃ¤Ã¤n,Lannoite,LannoitemÃ¤Ã¤rÃ¤
0,None,None,None,None,"TT Rhonia 30%, Tuure 30%, NN Ilmari 20 %",None,None
1,None,None,None,None,multamaan säilörehunurmi (K-maatalouden myymä ...,YaraMila Y20,420 kg/ha
2,mHtMr,None,None,None,"TT Grinstad 50%, Uula 20%, NN Valtteri 10%, RN...",YaraMila Y3,450 kg/ha
3,None,None,None,None,"TT 50%, NN 40%, eng.raeheinä 10%",None,26.4.-19
4,None,None,None,None,"raeheinä Riika 5%, nurminata klaara 5% puna-ap...",Yara Y4,20tn/ha


In [5]:
# Define the model you are using
llm = Ollama(model="llama3:latest")  # You can choose the model you want

df = extractValuesFromTextBatch(llm, test_sample, 5)
df

C:\Users\AIRAKSINENV\AppData\Local\Temp\ipykernel_81672\1359384189.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3:latest")  # You can choose the model you want


Batch 1 Results:
Here are the outputs in JSON format:

**Text 1**
{
    "Maalaji": "rm HHt",
    "Multavuus": "rm",
    "pH": None,
    "Nurmen ikä": None,
    "Mitä viljellään": "TT Nuutti 34 %, Grindstad 30 %, RN Swaj 13 %, NN Kasper 10 %, engl RH Birger 7 %, Mathilde 6%",
    "Lannoite": "YaraMila Y-25",
    "Lannoitemäärä": "440 kg/ha"
}

**Text 2**
{
    "Maalaji": "rm HiueS",
    "Multavuus": "rm",
    "pH": "6,4",
    "Nurmen ikä": "3 v. nurmi",
    "Mitä viljellään": "TT Nuutti 40%, NN Inkeri15%, RN Karoliina15%, E.rh Riikka15%,( Frida 5%+ Saija5%+Hebe 5%)",
    "Lannoite": None,
    "Lannoitemäärä": "100 kg N/ha"
}

**Text 3**
{
    "Maalaji": None,
    "Multavuus": None,
    "pH": None,
    "Nurmen ikä": None,
    "Mitä viljellään": "TT Nuutti 70 %, NN Inkeri 15 %, RN Karolina 15 %",
    "Lannoite": None,
    "Lannoitemäärä": None
}

**Text 4**
{
    "Maalaji": "m HtMr",
    "Multavuus": "m",
    "pH": None,
    "Nurmen ikä": "1v.nurmi",
    "Mitä viljellään": "TT Nuutti 45% 

,index,VUOSI,KUNTA,NAYTTEENOTTOPAIKAN NUMERO,LISATIETO_URL,INFOTEKSTI,AIKALEIMA,Maalaji,Multavuus,pH,Nurmen ikä,Mitä viljellään,Lannoite,Lannoitemäärä
0,526,2020,ISOKYRÖ,1,NaN,"Korjuu 17.6. TT Nuutti 34 %, Grindstad 30 %, R...",18.06.2020 14:36:10,None,None,None,None,None,None,None
1,309,2022,LOIMAA,1,NaN,"3 v. nurmi. rm HiueS. pH 6,4. Apilaseos mistä ...",09.06.2022 13:57:47,None,None,None,None,None,None,None
2,841,2018,ILMAJOKI,1,NaN,"Niitetty 9.6. TT Nuutti 70 %, NN Inkeri 15 %, ...",11.06.2018 20:17:00,None,None,None,None,None,None,None
3,166,2023,KUOPIO,3,NaN,"1v.nurmi. TT Nuutti 45% + Hertta 25%, NN Inker...",19.05.2023 12:44:28,None,None,None,None,None,None,None
4,843,2018,ILMAJOKI,3,NaN,"Voima ruokonataseos: TT Tenho 70%, NN Fure 10%...",30.05.2018 22:50:57,None,None,None,None,None,None,None


In [44]:
df = extractValuesFromTextBatch(llm, test_sample, 5)
df

Batch 1 Results:
Here are the extracted data for each text:

**Text 1**
Input: "2 v nurmi. rm Hiesumoreeni.  Hja klassikko TT 80% + NN 20%. Lannoitettu Y3  300 kg/ha."
Extracted Data:
Maalaji: HsMr
Multavuus: rm
pH: None
Nurmen ikä: 2 v
Mitä viljellään: Hja klassikko TT 80% + NN 20%
Lannoite: Y3
Lannoitemäärä: 300 kg/ha

**Text 2**
Input: "2 v. apilan.  rm HeS.  pH 6,6. TT Nuutti 40%, Erh Riikka 15%, NN inkeri 15%, RN Karoliina 15%,RehuM Juurlu 4% + eri apiloita 11%.  15.4. 100 kg N/ha. ei ruiskutettu, voikukkaa on.  ei täyd.kylvöä."
Extracted Data:
Maalaji: HeS
Multavuus: rm
pH: 6.6
Nurmen ikä: 2 v
Mitä viljellään: TT Nuutti 40%, Erh Riikka 15%, NN inkeri 15%, RN Karoliina 15%,RehuM Juurlu 4% + eri apiloita 11%
Lannoite: None
Lannoitemäärä: 100 kg N/ha

**Text 3**
Input: "2 LUOMU-nurmi. Karjanlantaa levitetty 2016 2 otteeseen. Siemeniä 28 kg/ha. Lajit on N.nata Radel, Ruokonata Barolex, Eng.raiheinä Barpasto ja 9% Intensive koiranheinää. Maalajit moreenista saviin."
Extracted Data:
Ma

,index,VUOSI,KUNTA,NAYTTEENOTTOPAIKAN NUMERO,LISATIETO_URL,INFOTEKSTI,AIKALEIMA,Maalaji,Multavuus,pH,Nurmen ikä,Mitä viljellään,Lannoite,Lannoitemäärä
0,390,2021,JUUPAJOKI,1,NaN,2 v nurmi. rm Hiesumoreeni. Hja klassikko TT ...,01.06.2021 10:21:23,None,None,None,None,None,None,None
1,186,2023,LOIMAA,1,NaN,"2 v. apilan. rm HeS. pH 6,6. TT Nuutti 40%, ...",01.06.2023 16:00:07,None,None,None,None,None,None,None
2,1036,2017,UUSIKAUPUNKI,1,NaN,2 v LUOMU-nurmi. Karjanlantaa levitetty 2016 2...,02.06.2017 13:51:36,None,None,None,None,None,None,None
3,787,2019,RANTASALMI,1,NaN,"TT Tuure 75%, NN Ilmari 10%, RN Karolina 15%",07.05.2019 11:14:24,None,None,None,None,None,None,None
4,268,2022,KAJAANI,1,NaN,"Näytepaikka koulutila. Luomu, 1v. nurmi. TT Tr...",03.06.2022 15:31:24,None,None,None,None,None,None,None
5,448,2021,PORNAINEN,1,NaN,NaN,01.06.2021 22:23:30,None,None,None,None,None,None,None
6,130,2023,HANKASALMI,1,NaN,"2v. nurmi. TT myöhäinen 35 % + aikainen 40 %, ...",22.05.2023 10:02:46,rmHe,rm,None,2v,"TT myöhäinen 35% + aikainen 40%, NN SW Minto 1...",YaraMila Y3,400 kg/ha
7,627,2020,PYHÄJÄRVI,1,NaN,"Timotei\tTuure\tLiete 25m3, YaraBela seleenisa...",04.06.2020 09:29:55,None,None,None,None,None,YaraBela seleenisalpietari,92 kg
8,107,2024,TORNIO,1,NaN,"Hankkija Pro, Kevät Y25 390kg/ha ja kesällä li...",28.05.2024 10:39:44,None,None,None,2 vuoden nurmi,"Hankkija Pro, Kevät Y25 390kg/ha ja kesällä li...",None,None
9,445,2021,PEDERSÖRE,1,NaN,"Näytepaikka Purmo. TT Nuutti 40%, NN Inkeri 15...",08.06.2021 16:11:52,None,None,None,None,"TT Nuutti 40%, NN Inkeri 15%, RN Karolina 15%,...",Cemagro (27-0-0-Mg-Ca),150 kg/ha
